In [2]:
import numpy as np
import nltk
# nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import porter
from nltk.text import TextCollection
import pandas as pd
import sys
import os
import json
import string
import re
from scipy import spatial
from heapq import nlargest

from data_loader import DataLoader

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lky\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
loader1 = DataLoader(DataLoader.data_path1)
table1 = loader1.load_table()

loader2 = DataLoader(DataLoader.data_path2)
table2 = loader2.load_table()
table = pd.concat([table1, table2], ignore_index = True)
table

d:\study\NTU_LEARN\AI6122 TEXT MANAGEMENT & PROCESSING\project\data_loader.py:13: FutureWarning: Starting with pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only.
  self.table = pd.read_json(data_path, 'records', lines = True);
d:\study\NTU_LEARN\AI6122 TEXT MANAGEMENT & PROCESSING\project\data_loader.py:13: FutureWarning: Starting with pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only.
  self.table = pd.read_json(data_path, 'records', lines = True);


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AIVZMEFEAGF3,B0090UTDKO,txlucky1,"[2, 2]",Although it would seem that most of the inform...,5,Surprisingly useful advice,1367280000,"04 30, 2013"
1,A17O3IALB0VVSA,B00ASFHQUY,mapleleafmanny,"[2, 2]",I've always loved the story of Peter Pan and I...,4,Better then the classic story,1379635200,"09 20, 2013"
2,A1Q6090OWANRMC,B00B6EFPEK,Bev Seabrook,"[18, 19]",Most people equate being frugal with being dep...,5,Start Living a Better Life Today,1359590400,"01 31, 2013"
3,A171L7UJTO7HXU,B00AQJRYWC,Ro13135,"[0, 0]","Good read, funny at times but author wrapped u...",4,Good Book but disapointing ending,1405728000,"07 19, 2014"
4,A2ISDBOVMRCRLO,B00ARY7ZUC,"C. L. Orlando ""always reading""","[0, 0]",I like the realistic viewpoints of the heroine...,4,Liking this series very much,1373846400,"07 15, 2013"
...,...,...,...,...,...,...,...,...,...
395,A3AL8GQ69QE7WN,B0014XCMVM,Jason Stein,"[2, 3]",Alanis Morissette always makes decent albums. ...,4,I have been missing the rapture this whole time.,1215734400,"07 11, 2008"
396,A1J5KCZC8CMW9I,B000B8QF00,"H3@+h ""Over 1500 reviews!""","[2, 3]","There are some who may still think of ""Jagged ...",4,"Almost ""Everything"" of hers.",1132012800,"11 15, 2005"
397,A1G7F9ZDJ07EMA,B00003NHAR,John,"[1, 2]",Well there is not much to say about an album t...,1,Volume 3,1069027200,"11 17, 2003"
398,AGZ28CC1MLUAD,B0000024JN,thaman,"[1, 3]","YEAH BRING BACK SOME GOOD TIMES,WHEN I WAS A L...",4,MY FIRST HIP HOP CASSET TAPE!,1133136000,"11 28, 2005"


# Preprocess

In [3]:
def preprocess(doc):
    # convert to lower cases
    doc = doc.lower()
    
    # remove stopwords
    stop_words = stopwords.words('english')
    word_tokens = word_tokenize(doc)
    doc = [w for w in word_tokens if w not in stop_words]
    
    # remove punctuations
    doc = [w for w in doc if w.isalnum()]
    
    # stem
    stemmer = porter.PorterStemmer()
    doc = [stemmer.stem(w) for w in doc]
    
    return doc

# Collecting Documents

In [4]:
# Assume research can be performed on [reviewText] and [summary] / [reviewerID] and [asin]
# collection of [reviewText] and [summary]
text_collection = []
size = len(table)
for i in range(size):
    text_collection.append(preprocess(table.iloc[i]['reviewText'] + ' ' + table.iloc[i]['summary']))
    
# collection of [reviewerID] and [asin]
id_collection = {}

for i in range(size):
    if table.iloc[i]['reviewerID'].lower() not in id_collection.keys():
        id_collection[table.iloc[i]['reviewerID'].lower()] = [i]
    else:
        id_collection[table.iloc[i]['reviewerID'].lower()].append(i)
    if table.iloc[i]['asin'].lower() not in id_collection.keys():
        id_collection[table.iloc[i]['asin'].lower()] = [i]
    else: 
        id_collection[table.iloc[i]['asin'].lower()].append(i)

# TF-IDF

# corpus
corpus = TextCollection(text_collection)

# tf-idf
vs = np.empty([len(corpus),size])
for i in range(size):
    for j, term in enumerate(corpus):
        vs[j][i] = corpus.tf_idf(term, text_collection[i])

# Cosine Similarity

In [11]:
def cosSim(qvector, qterm):
    h = []
    all_zeros = not np.any(qvector)
    for i in range(size):
        if all_zeros:
            h.append(0)
        else:
            h.append(1- spatial.distance.cosine(vs[:,i], qvector))
            
    # Add weight to [reviewerID] and [asin] in query, if exist
    for term in qterm:
        if term in id_collection.keys():
            for i in id_collection[term]:
                h[i] += 1
    return h

# Query

In [12]:
# Set N = 10 (Top N results)
N = 10
qvector = np.zeros(len(corpus))

# return the indices of N largest items
def result(N):
    query = input("Research for:")
    qterm = preprocess(query)
    
    for i, term in enumerate(corpus):
        qvector[i] = corpus.tf_idf(term, qterm)
    
    h = cosSim(qvector, qterm)
    if not np.any(h):
        return "Your search - " + query + " - did not match any documents."
        
    else:
        indices = map(h.index, nlargest(N,h))
        return table.loc[indices,:]

In [13]:
result(N)

Research for:B00ARY7ZUC realistic


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
4,A2ISDBOVMRCRLO,B00ARY7ZUC,"C. L. Orlando ""always reading""","[0, 0]",I like the realistic viewpoints of the heroine...,4,Liking this series very much,1373846400,"07 15, 2013"
100,A1IK8S27LLFMX7,B007R5T3SQ,dmd,"[0, 0]",The story line was well thought out and was ea...,5,Wonderful book!,1395273600,"03 20, 2014"
6,A13QTZ8CIMHHG4,B003SE7K4O,"LAS Reviewer ""The Long and the Short Of It Re...","[3, 3]",Originally posted at: [...]You never know how ...,5,Wonderful Read!,1282176000,"08 19, 2010"
178,AJ9KKPHO6E6KU,B006PI0Q2I,Angel Wing Buyer,"[1, 1]",Gary Henry's literary craftsmanship is stunnin...,5,Brilliant & Magical Brain Candy--a Stunning Co...,1334102400,"04 11, 2012"
47,A2VMDDSTBHTDS0,B00HY0CBII,Yazi,"[0, 0]",Hmm&#8230;this book was very&#8230;..different...,3,Definitely very different from the prequel,1390608000,"01 25, 2014"
0,AIVZMEFEAGF3,B0090UTDKO,txlucky1,"[2, 2]",Although it would seem that most of the inform...,5,Surprisingly useful advice,1367280000,"04 30, 2013"
0,AIVZMEFEAGF3,B0090UTDKO,txlucky1,"[2, 2]",Although it would seem that most of the inform...,5,Surprisingly useful advice,1367280000,"04 30, 2013"
0,AIVZMEFEAGF3,B0090UTDKO,txlucky1,"[2, 2]",Although it would seem that most of the inform...,5,Surprisingly useful advice,1367280000,"04 30, 2013"
0,AIVZMEFEAGF3,B0090UTDKO,txlucky1,"[2, 2]",Although it would seem that most of the inform...,5,Surprisingly useful advice,1367280000,"04 30, 2013"
0,AIVZMEFEAGF3,B0090UTDKO,txlucky1,"[2, 2]",Although it would seem that most of the inform...,5,Surprisingly useful advice,1367280000,"04 30, 2013"
